In [ ]:
#!/usr/bin/env python
# coding: utf-8
from sentence_transformers import SentenceTransformer, LoggingHandler, losses, util
from sentence_transformers.datasets import SentenceLabelDataset
from torch.utils.data import DataLoader
from sentence_transformers.readers import InputExample
from sentence_transformers.evaluation import TripletEvaluator
from datetime import datetime
import logging
import os
import random
from collections import defaultdict
import scipy.spatial
import numpy as np
import os, json
import glob
import re
import torch
import pandas as pd
from sentence_transformers import SentenceTransformer, util
from tokenizers import Tokenizer
from datetime import datetime
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn.metrics import classification_report
from sklearn.metrics import multilabel_confusion_matrix
torch.manual_seed(1)
import pandas as pd
import torch
import random
import itertools
import pickle
import time


top_claim_order = []
top_claim_ids = []
top_similarity_scores = []
text_list = []
id_list = []
cosine_similarity_list = []
save_F1_list = []
finetune_score_list = []
text_list_min = []
id_list_min = []
cosine_similarity_list_min = []
save_F1_list_min = []
finetune_score_list_min = []
df_claim_cpc_test_list = []
df_claim_cpc_id_list = []


#Load sentences & embeddings from disc
with open('/home/ubuntu/storage_data_new/data_old_strato_volume/patent_data_root/df_claim_test_1M_pre_duplicates_removed_663.pkl', "rb") as fIn:
    stored_data = pickle.load(fIn)
    stored_patent_test_embeddings_id = stored_data['patent_id']
    test_embeddings = stored_data['claim_embeddings']

with open('/home/ubuntu/storage_data_new/data_old_strato_volume/patent_data_root/df_claim_train_1M_pre_duplicates_removed_663.pkl', "rb") as fIn:
    stored_data = pickle.load(fIn)
    stored_patent_train_embeddings_id = stored_data['patent_id']
    claim_embeddings = stored_data['claim_embeddings']
    
df_claim_cpc_test = pd.read_csv('/home/ubuntu/storage_data_new/data_old_strato_volume/patent_data_root/df_claim_test_1M_pre_duplicates_removed_663.csv', encoding='ISO-8859-1')
df_claim_cpc_train = pd.read_csv('/home/ubuntu/storage_data_new/data_old_strato_volume/patent_data_root/df_claim_train_1M_pre_duplicates_removed_663.csv', encoding='ISO-8859-1')

claims = list(df_claim_cpc_train.text)
patent_id = list(df_claim_cpc_train.id)


text_list = []
id_list = []
cosine_similarity_list = []
save_F1_list = []
finetune_score_list = []
df_claim_cpc_test_list = []
df_claim_cpc_id_list = []

def get_search_hit (tet_embeddings):
    top_claim_order = []
    top_claim_ids = []
    top_similarity_scores = []
    search_hits = util.semantic_search(np.array(tet_embeddings).reshape(1,-1), claim_embeddings, 10000, 5000000, 10)
    for item in range(len(search_hits[0])):
        top_claim_order = search_hits[0][item].get('corpus_id')
        top_claim_ids.append(stored_patent_train_embeddings_id[top_claim_order])
        top_similarity_scores.append(search_hits[0][item].get('score'))    
        top_100_similar_patents_df = pd.DataFrame({
           'top_claim_ids': top_claim_ids,
           'cosine_similarity': top_similarity_scores,
        })
    return top_100_similar_patents_df

for patent_id in range(len(test_embeddings[:100])):
    top_100_similar_patents_df = get_search_hit(test_embeddings[patent_id])
    result = pd.DataFrame()
    result = pd.merge(top_100_similar_patents_df, df_claim_cpc_train, left_on='top_claim_ids',right_on='id',how='left',suffixes=('_left','_right'))
    top_100_similar_patents_df = pd.DataFrame(columns = top_100_similar_patents_df.columns)
    save_F1 = []
    

    for item in range(len(result)):
        data = torch.tensor((result.iloc[item:item+1, 7:].to_numpy()).astype(float), dtype=torch.float32)
        output_df = pd.DataFrame(data, columns=result.columns[7:]).astype(float)
        y_pred = output_df.iloc[:, :].to_numpy()
        y_true = df_claim_cpc_test.iloc[patent_id:patent_id + 1, 5:].to_numpy()
        temp = 0
        for i in range(y_true.shape[0]):
            if (sum(y_true[i]) == 0) and (sum(y_pred[i]) == 0):
                continue
            temp_save = (2*sum(np.logical_and(y_true[i], y_pred[i])))/ (sum(y_true[i])+sum(y_pred[i]))
            save_F1.append(temp_save)
            temp += temp_save
    result['save_F1'] = save_F1
    result['finetune_score'] = save_F1
    for item in range(len(result)):
        if result['save_F1'][item] >= 0.80:
            if result['cosine_similarity'][item] * 1.2 > 1:
                result['finetune_score'][item] = 1
            else:
                result['finetune_score'][item] = result['cosine_similarity'][item] * 1.2
        if result['save_F1'][item] > 0.60 and result['save_F1'][item] < 0.80:
            if result['cosine_similarity'][item] * 1.1 > 1:
                result['finetune_score'][item] = 1
            else:
                result['finetune_score'][item] = result['cosine_similarity'][item] * 1.1
        if result['save_F1'][item] > 0 and result['save_F1'][item] <= 0.60:
            result['finetune_score'][item] = result['cosine_similarity'][item]
        if result['save_F1'][item] == 0:
            result['finetune_score'][item] = result['cosine_similarity'][item] * 0.7

        
    df_claim_cpc_test_list.append(df_claim_cpc_test['text'][patent_id])
    df_claim_cpc_id_list.append(df_claim_cpc_test['id'][patent_id])
    text_list.append(result['text'][result['finetune_score'].idxmax()])
    id_list.append(result['id'][result['finetune_score'].idxmax()])
    cosine_similarity_list.append(result['cosine_similarity'][result['finetune_score'].idxmax()])
    save_F1_list.append(result['save_F1'][result['finetune_score'].idxmax()])
    finetune_score_list.append(result['finetune_score'].max())
    text_list_min.append(result['text'][result['finetune_score'].idxmin()])
    id_list_min.append(result['id'][result['finetune_score'].idxmin()])
    cosine_similarity_list_min.append(result['cosine_similarity'][result['finetune_score'].idxmin()])
    save_F1_list_min.append(result['save_F1'][result['finetune_score'].idxmin()])
    finetune_score_list_min.append(result['finetune_score'].min())

    
df_top_1_patents_finetuning = pd.DataFrame({
           'claim_1': df_claim_cpc_test_list,
           'claim_1_id': df_claim_cpc_id_list,
           'claim_2': text_list,
           'claim_2_id': id_list,
           'cosine_similarity_2': cosine_similarity_list,
           'save_F1_lis_2': save_F1_list,
           'finetune_score_list_2': finetune_score_list,
           'claim_3': text_list_min,
           'claim_3_id': id_list_min,
           'cosine_similarity_3': cosine_similarity_list_min,
           'save_F1_lis_3': save_F1_list_min,
           'finetune_score_list_3': finetune_score_list_min
        })
df_top_1_patents_finetuning = df_top_1_patents_finetuning[df_top_1_patents_finetuning['save_F1_lis_3']==0]


def triplets_from_labeled_dataset(df_top_1_patents_finetuning):
    triplets = []
    for instance in range(len(df_top_1_patents_finetuning)):
        triplets.append(InputExample(texts=[df_top_1_patents_finetuning.claim_1[0], df_top_1_patents_finetuning.claim_2[0], df_top_1_patents_finetuning.claim_3[0]]))
    return triplets



# You can specify any huggingface/transformers pre-trained model here, for example, bert-base-uncased, roberta-base, xlm-roberta-base
model_name = 'flax-sentence-embeddings/all_datasets_v3_mpnet-base'

### Create a torch.DataLoader that passes training batch instances to our model
train_batch_size = 32
output_path = (
    "/home/ubuntu/storage_data_new/data_old_strato_volume/finetuningPatentSBERT/model_test/finetune-batch-hard-triple-new-2022-"
    + model_name
    + "-"
    + datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
)
num_epochs = 2

logging.info("Loading dataset")
train_set = triplets_from_labeled_dataset(df_top_1_patents_finetuning)

# We create a special dataset "SentenceLabelDataset" to wrap out train_set
# It will yield batches that contain at least two samples with the same label
train_data_sampler = SentenceLabelDataset(train_set)
train_dataloader = DataLoader(train_data_sampler, batch_size=32)


# Load pretrained model
logging.info("Load model")
model = SentenceTransformer(model_name)


### Triplet losses ####################
### There are 4 triplet loss variants:
### - BatchHardTripletLoss
### - BatchHardSoftMarginTripletLoss
### - BatchSemiHardTripletLoss
### - BatchAllTripletLoss
#######################################


train_loss = losses.BatchHardSoftMarginTripletLoss(model=model)


# Train the model
model.fit(train_objectives=[(train_dataloader, train_loss)], epochs=1, warmup_steps=100)
model.save(output_path)
